Import warnings and ignore them 

In [14]:
import warnings
warnings.filterwarnings('ignore')

Clone the official github repo of **YOLOv9** model

In [15]:
!git clone https://github.com/WongKinYiu/yolov9.git

Cloning into 'yolov9'...
remote: Enumerating objects: 713, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 713 (delta 250), reused 202 (delta 201), pack-reused 374
Receiving objects: 100% (713/713), 3.24 MiB | 16.32 MiB/s, done.
Resolving deltas: 100% (295/295), done.


Install gdown package to download DATASET from my google drive

In [6]:
!conda install -y gdown

Retrieving notices: ...working... done
done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdown


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2024.6.2   |       hbcca054_0         152 KB  conda-forge
    filelock-3.14.0            |     pyhd8ed1ab_0          16 KB  conda-forge
    gdown-5.2.0                |     pyhd8ed1ab_0          21 KB  conda-forge
    openssl-3.3.0              |       h4ab18f5_3         2.8 MB  conda-forge
    ------------------------------------------------------------
                       

Download dataset to kaggle notebook using the share link of the google drive

In [28]:
%cd /kaggle/working/
!gdown 1-Esam-6_haIP1nps3PPKq0icvEkX3QiV

/kaggle/working
Downloading...
From (original): https://drive.google.com/uc?id=1-Esam-6_haIP1nps3PPKq0icvEkX3QiV
From (redirected): https://drive.google.com/uc?id=1-Esam-6_haIP1nps3PPKq0icvEkX3QiV&confirm=t&uuid=f7b5f34e-0668-435f-8f84-085313c47bea
To: /kaggle/working/bdsl_final.zip
100%|██████████████████████████████████████| 5.85G/5.85G [01:26<00:00, 67.3MB/s]


Unzip the dataset to the yolov9 repository's  directory

In [35]:
import zipfile
import os



# Step 2: Define the path to the zip file
zip_file_path = r'/kaggle/working/bdsl_final.zip'

# Step 3: Extract the name of the zip file (without extension) to create a folder
zip_file_name = os.path.basename(zip_file_path)
folder_name = os.path.splitext(zip_file_name)[0]
extract_to_path = os.path.join('/kaggle/working/yolov9', folder_name)

# Step 4: Create the output directory if it doesn't exist
os.makedirs(extract_to_path, exist_ok=True)

# Step 5: Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

# Verify the extraction
if os.path.exists(extract_to_path):
    print(f"Folder successfully unzipped to {extract_to_path}")
else:
    print("Failed to unzip the folder.")

Folder successfully unzipped to /kaggle/working/yolov9/bdsl_final


In [ ]:
Remove the zipped dataset

In [36]:
import os

file_path = '/kaggle/working/bdsl_final.zip'

# Check if the file exists before trying to delete it
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been deleted")
else:
    print(f"{file_path} does not exist")


/kaggle/working/bdsl_final.zip has been deleted


Download customized yml file for dataset path in the repository folder

In [ ]:
!gdown 1LWWlVuwK6kQl9DVOAUK6kFVxhGKRFgMg

Install ultralytics packages 

In [2]:
!pip install ultralytics 


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 764.8 kB/s eta 0:00:000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 6.3 MB/s eta 0:00:00a 0:00:01


Import the model and show model details

In [9]:
from ultralytics import YOLO


# Build a YOLOv9c model from pretrained weight
model = YOLO("Yolov9.pt")

# Display model information (optional)
model.info()



YOLOv9c summary: 618 layers, 25567011 parameters, 0 gradients, 103.9 GFLOPs


(618, 25567011, 0, 103.8893568)

Define DataParallelModel class for multi gpu training

In [10]:
import torch
import torch.nn as nn


class DataParallelModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.block1 = nn.Linear(10, 20)

        # wrap block2 in DataParallel
        self.block2 = nn.Linear(20, 20)
        self.block2 = nn.DataParallel(self.block2)

        self.block3 = nn.Linear(20, 20)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        return x

Setup WandB for saving model training progress

In [ ]:

import wandb
import random  # for demo script
WANDB_API_KEY='957d38e93afcd97d0286c494f525efdb6caf7707'

wandb.login(key=WANDB_API_KEY)

epochs = 20
lr = 0.01

run = wandb.init(
    # Set the project where this run will be logged
    project="Bangla Sign Language Detection",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": lr,
        "epochs": epochs,
    },
)


Train the model with custom dataset

In [ ]:
# Train the model on the custom dataset for 20 epochs
results = model.train(data="/kaggle/working/yolov9/data.yaml",  epochs=20, imgsz=640, verbose=True,save=True,save_period=1,device=[0,1])

WandB code for training accuracy and learning rate

In [ ]:
offset = random.random() / 5
print(f"lr: {lr}")

# simulating a training run
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset
    print(f"epoch={epoch}, accuracy={acc}, loss={loss}")
    wandb.log({"accuracy": acc, "loss": loss})

run.log_code()